In [74]:
import pandas as pd
import numpy as np
import json
import re
import folium

In [75]:
population = pd.read_excel('./data/서울 읍면동 인구.xls',skiprows=1)
population

,기간,자치구,동,세대,합계,합계.1,합계.2,한국인,한국인.1,한국인.2,외국인,외국인.1,외국인.2,세대당인구,65세이상고령자
0,기간,자치구,동,세대,계,남자,여자,계,남자,여자,계,남자,여자,세대당인구,65세이상고령자
1,2022.1/4,합계,합계,4451444,9732617,4716704,5015913,9506778,4613910,4892868,225839,102794,123045,2.14,1628980
2,2022.1/4,종로구,소계,73766,153684,74018,79666,144275,69911,74364,9409,4107,5302,1.96,28064
3,2022.1/4,종로구,사직동,4679,9630,4335,5295,9370,4189,5181,260,146,114,2,1787
4,2022.1/4,종로구,삼청동,1227,2741,1312,1429,2441,1168,1273,300,144,156,1.99,635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,2022.1/4,강동구,천호2동,17499,33671,16467,17204,33070,16193,16877,601,274,327,1.89,5978
449,2022.1/4,강동구,길동,21917,45863,22374,23489,45554,22269,23285,309,105,204,2.08,8167
450,2022.1/4,강동구,강일동,12889,32947,15789,17158,32762,15733,17029,185,56,129,2.54,5437
451,2022.1/4,강동구,상일1동,13477,38832,19137,19695,38713,19082,19631,119,55,64,2.87,4561


In [76]:
raw_seoul_population = population.iloc[3:,2:4]
raw_seoul_population

,동,세대
3,사직동,4679
4,삼청동,1227
5,부암동,4242
6,평창동,7493
7,무악동,3071
...,...,...
448,천호2동,17499
449,길동,21917
450,강일동,12889
451,상일1동,13477


In [77]:
no_data = raw_seoul_population[raw_seoul_population['동'] == '소계'].index
no_data

Int64Index([ 20,  36,  53,  71,  87, 102, 119, 140, 154, 169, 189, 206, 221,
            238, 257, 278, 295, 306, 325, 341, 363, 382, 405, 433],
           dtype='int64')

In [78]:
seoul_population = raw_seoul_population.drop(no_data)

In [79]:
seoul_population

,동,세대
3,사직동,4679
4,삼청동,1227
5,부암동,4242
6,평창동,7493
7,무악동,3071
...,...,...
448,천호2동,17499
449,길동,21917
450,강일동,12889
451,상일1동,13477


In [80]:
seoul_population.columns=['name','values']

In [81]:
seoul_population

,name,values
3,사직동,4679
4,삼청동,1227
5,부암동,4242
6,평창동,7493
7,무악동,3071
...,...,...
448,천호2동,17499
449,길동,21917
450,강일동,12889
451,상일1동,13477


In [93]:
geo_json='./data/Seoul_geoJson.geojson'
seoul_population.sort_values(by='name')
#seoul_population['name'] = seoul_population['name'].apply(lambda x: re.compile('[가-힣]+').findall(x)[0])

,name,values
421,가락1동,9865
422,가락2동,12359
420,가락본동,11837
290,가리봉동,6112
296,가산동,17076
...,...,...
41,효창동,4407
37,후암동,8472
89,휘경1동,7769
90,휘경2동,12231


In [94]:
with open(geo_json,'r',encoding='utf-8') as f:
    seoul_geo = f.read()

In [95]:
seoul_geo

'{\n"type": "FeatureCollection",\n"name": "Seoul_geoJson",\n"features": [\n{ "type": "Feature", "properties": { "EMD_CD": "11290129", "EMD_NM": "보문동7가", "SGG_OID": 7, "COL_ADM_SE": "11290", "GID": 5222, "x": 127.0223774511, "y": 37.579774084999997 }, "geometry": { "type": "Point", "coordinates": [ 957827.73496828333009, 1953483.930514485575259 ] } },\n{ "type": "Feature", "properties": { "EMD_CD": "11290103", "EMD_NM": "돈암동", "SGG_OID": 8, "COL_ADM_SE": "11290", "GID": 5223, "x": 127.0182843078, "y": 37.597796163200002 }, "geometry": { "type": "Point", "coordinates": [ 957476.578209447441623, 1955485.241178313735873 ] } },\n{ "type": "Feature", "properties": { "EMD_CD": "11290139", "EMD_NM": "석관동", "SGG_OID": 9, "COL_ADM_SE": "11290", "GID": 5224, "x": 127.0614872235, "y": 37.608269478399997 }, "geometry": { "type": "Point", "coordinates": [ 961295.759549911366776, 1956628.521524442359805 ] } },\n{ "type": "Feature", "properties": { "EMD_CD": "11290114", "EMD_NM": "삼선동4가", "SGG_OID": 2

In [102]:
m = folium.Map(
    location=[37.566345, 126.977893],
    tiles = 'cartodbpositron'
)

folium.Choropleth(
    geo_data = seoul_geo,
    name = 'choropleth',
    data = seoul_population,
    columns = ['name','values'],
    key_on = 'feature.properties.EMD_NM',
    fill_color='PuBu',
    fill_opacity=0.7,
    line_opacity=0.2
).add_to(m)

In [103]:
m